# Home Assignment Non Neural Machine Learning
## Alireza Bolhassani
----
At each stage, the required libraries will be imported.

Following cell imports the pandas libraries.

In [94]:
import pandas as pd

-----
Load data in a variable

In [95]:
df = pd.read_csv("data.csv")
df.head(3)

,Data As Of,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths,Number of Mentions,Flag
0,06/25/2023,01/01/2020,06/24/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,1554.0,1630.0,NaN
1,06/25/2023,01/01/2020,06/24/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,5775.0,5998.0,NaN
2,06/25/2023,01/01/2020,06/24/2023,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,15026.0,15643.0,NaN


---
Restrict the dataset to the rows where Start Date is "01/01/2020" and the End Date is "06/24/2023".
We convert the 'Start Date' and 'End Date' columns to datetime objects using pd.to_datetime() to ensure proper date comparisons.

In [96]:
df["Data As Of"] = pd.to_datetime(df["Data As Of"], format="%m/%d/%Y")
df["Start Date"] = pd.to_datetime(df["Start Date"], format="%m/%d/%Y")
df["End Date"] = pd.to_datetime(df["End Date"])

df = df.loc[df["Start Date"] > "01/01/2020"]
df = df.loc[df["End Date"] < "06/24/2023"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 521640 entries, 12421 to 583738
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Data As Of          521640 non-null  datetime64[ns]
 1   Start Date          521640 non-null  datetime64[ns]
 2   End Date            521640 non-null  datetime64[ns]
 3   Group               521640 non-null  object        
 4   Year                521640 non-null  float64       
 5   Month               496800 non-null  float64       
 6   State               521640 non-null  object        
 7   Condition Group     521640 non-null  object        
 8   Condition           521640 non-null  object        
 9   ICD10_codes         521640 non-null  object        
 10  Age Group           521640 non-null  object        
 11  COVID-19 Deaths     359775 non-null  float64       
 12  Number of Mentions  364940 non-null  float64       
 13  Flag                161865 non

---
Remove rows not belonging to specific age groups.

In [97]:
df = df.loc[df["Age Group"] != "Not stated"]
df["Age Group"].unique()

array(['0-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84',
       '85+', 'All Ages'], dtype=object)

---
Convert Covid deaths rows from float to int and fill empty cell with 0.

In [98]:
df["COVID-19 Deaths"] = df["COVID-19 Deaths"].fillna(0)
df["COVID-19 Deaths"] = df["COVID-19 Deaths"].astype("int")

---
Remove rows with the United States as State.

In [99]:
df = df.loc[df["State"] != "United States"]
df["State"].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'New York City',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

---
Investigate missing values, and deal with them if you find any.   
If a row contains missing values and it's not feasible to impute them, you can drop the entire row.


In [29]:
# df.dropna(inplace=True)
# df['Column_Name'].fillna(0, inplace=True)

---
Encode age groups as you see fit as numeric columns

In [100]:
# Define a mapping from age groups to numeric values
age_group_mapping = {
    "0-24": 1,
    "25-34": 2,
    "35-44": 3,
    "45-54": 4,
    "55-64": 5,
    "65-74": 6,
    "75-84": 7,
    "85+": 8,
    "All Ages": 9,
}

# Create a new column with numeric values based on the mapping
df["Age_Group_Numeric"] = df["Age Group"].map(age_group_mapping)

df["Age_Group_Numeric"].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

---
Create a Broad Condition Group variable, which should be the same as the Condition Group variable for the two most frequent condition groups, but has the value "other" for all other condition groups.



In [101]:
# Find the two most frequent condition groups
top_condition_groups = df['Condition Group'].value_counts().nlargest(2).index

# Create a new column 'Broad Condition Group' based on the condition
df['Broad Condition Group'] = df['Condition Group'].apply(lambda x: x if x in top_condition_groups else 'other')


df["Broad Condition Group"].unique()

array(['Respiratory diseases', 'Circulatory diseases', 'other'],
      dtype=object)

---
Label-encode the Broad Condition Group variable.

In [102]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()


# Fit and transform the "Broad Condition Group" column
df["Broad Condition Group Encoded"] = label_encoder.fit_transform(
    df["Broad Condition Group"]
)

---
---
## Classification
Below cell, imports the train_test_split function from the sklearn.model_selection module.

In the following, the train_test_split function from scikit-learn's model_selection module is used to split the input data and target data into training and testing sets. The function takes four arguments:
    input_data: the input variables used for training and testing the model.
    target_data: the target variable used for training and testing the model.
    test_size: the proportion of the data to be used for testing. In this case, 33% of the data is used for   testing, while 67% is used for training.
    random_state: a seed for the random number generator used for splitting the data. This ensures that the same split can be obtained every time the code is run. In this case, the random state is set to 40.
The function returns four variables: X_train, X_test, y_train, and y_test. The X_train and y_train variables represent the training set, while the X_test and y_test variables represent the testing set. These variables can be used to train and evaluate machine learning models.

In [103]:
from sklearn.model_selection import train_test_split

X = df.loc[:,["COVID-19 Deaths","Age_Group_Numeric"]]
y = df["Broad Condition Group Encoded"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X.head(3)

,COVID-19 Deaths,Age_Group_Numeric
13341,13,1
13342,0,1
13345,71,2


In the following, it shows how many rows and columns each set has. The first two print statements show the shape of the X_train and X_test sets, while the last two print statements show the shape of the y_train and y_test sets, respectively.

In [104]:
print("Training set:", X_train.shape)
print("Test set:", X_test.shape)
print("Training set:", y_train.shape)
print("Test set:", y_test.shape)

Training set: (368625, 2)
Test set: (92157, 2)
Training set: (368625,)
Test set: (92157,)


---
## Logistic Regression

In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the Logistic Regression model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_regression_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Logistic Regression Model:")
print(f"Accuracy: {accuracy:.2f}")
print(report)

Logistic Regression Model:
Accuracy: 0.43
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     28219
           1       0.00      0.00      0.00     23900
           2       0.43      1.00      0.61     40038

    accuracy                           0.43     92157
   macro avg       0.14      0.33      0.20     92157
weighted avg       0.19      0.43      0.26     92157



/Users/user/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/user/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/user/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---
## Random Forest Classifier

In [106]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest Classifier model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = random_forest_model.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print("\nRandom Forest Classifier Model:")
print(f"Accuracy: {accuracy_rf:.2f}")
print(report_rf)


Random Forest Classifier Model:
Accuracy: 0.43
              precision    recall  f1-score   support

           0       0.30      0.02      0.03     28219
           1       0.29      0.01      0.03     23900
           2       0.43      0.97      0.60     40038

    accuracy                           0.43     92157
   macro avg       0.34      0.33      0.22     92157
weighted avg       0.36      0.43      0.28     92157



In [107]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = sqrt(mse)
print('Random Forest MSE:', mse)
print('Random Forest MAE:', mae)
print('Random Forest RMSE:', rmse)

Random Forest MSE: 1.4841628959276019
Random Forest MAE: 0.8717514676041972
Random Forest RMSE: 1.218262244316716
